In [1]:
from __future__ import division, print_function, unicode_literals
import sys
from collections import defaultdict

from tqdm import tqdm
import numpy as np
import pandas as pd
import joblib

import transformers
from transformers import AutoModel, AutoTokenizer

import torch
import torch.nn as nn
import torch.utils.data
from torchcrf import CRF

from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import classification_report

In [2]:
data_folder = '/media/lurker18/Local Disk/BioNER-Abbrev/Dataset/NCBI'

In [3]:
model_name = "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract"

pubmedbert = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = False)
pubmedbert.save_pretrained(f"./pretrained_model/pubmedbert")
tokenizer.save_pretrained("./pretrained_model/pubmedbert")

[2023-12-05 14:01:51,092] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


('./pretrained_model/pubmedbert/tokenizer_config.json',
 './pretrained_model/pubmedbert/special_tokens_map.json',
 './pretrained_model/pubmedbert/vocab.txt',
 './pretrained_model/pubmedbert/added_tokens.json')

In [4]:
# Hyperparameter Settings
device = "cuda:0"
max_len = 256
train_batch_size = 64
valid_batch_size = 64
epochs = 30
num_workers = 8
BASE_MODEL_PATH = './pretrained_model/pubmedbert'
model_path = "models/NBCI_pubmedbert"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BASE_MODEL_PATH,
                                                       do_lower_case = True)

In [5]:
class EntityDataset:
    def __init__(self, texts, tags,enc_tag):
        self.texts = texts
        self.tags = tags
        self.enc_tag = enc_tag
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        tags = self.tags[item]

        ids = []
        target_tag =[]

        for i, s in enumerate(text):
            inputs = TOKENIZER.encode(
                str(s),
                add_special_tokens = False
            )
            input_len = len(inputs)
            ids.extend(inputs)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:max_len - 2]
        target_tag = target_tag[:max_len - 2]

        ids = [102] + ids + [103]
        o_tag = self.enc_tag.transform(["O"])[0]
        target_tag = [o_tag] + target_tag + [o_tag]

        mask = [1] * len(ids)
        token_type_ids = [0] * len(ids)

        padding_len = max_len - len(ids)

        ids = ids + ([0] * padding_len)
        mask = mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tag = target_tag + ([0] * padding_len)

        return {
            "ids": torch.tensor(ids, dtype = torch.long),
            "mask": torch.tensor(mask, dtype = torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }

In [6]:

class EntityModel(nn.Module):
    def __init__(self, num_tag):
        super(EntityModel, self).__init__()
        self.num_tag = num_tag
        self.bert = transformers.BertModel.from_pretrained(BASE_MODEL_PATH, return_dict = False)
        self.bilstm =  nn.LSTM(768, 1024 // 2, num_layers = 1, bidirectional = True, batch_first = True)

        self.dropout_tag = nn.Dropout(0.3)
        
        self.hidden2tag_tag = nn.Linear(1024, self.num_tag)

        self.crf_tag = CRF(self.num_tag, batch_first = True)
    
    
    # return the loss only, not encode the tag
    def forward(self, ids, mask, token_type_ids, target_tag):
        x, _ = self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids)
        h, _ = self.bilstm(x)

        o_tag = self.dropout_tag(h)
        tag = self.hidden2tag_tag(o_tag)
        mask = torch.where(mask == 1, True, False)

        loss_tag = - self.crf_tag(tag, 
                                  target_tag, 
                                  mask = mask, 
                                  reduction = 'token_mean')
        loss = loss_tag
        
        return loss


    # encode the tag, dont return loss
    def encode(self, ids, mask, token_type_ids, target_tag):
        # Bert - BiLSTM
        x, _ = self.bert(ids, 
                         attention_mask = mask, 
                         token_type_ids = token_type_ids)
        h, _ = self.bilstm(x)

        # drop out
        o_tag = self.dropout_tag(h)
        # o_pos = self.dropout_pos(h)

        # Hidden2Tag (Linear)
        tag = self.hidden2tag_tag(o_tag)
        mask = torch.where(mask == 1, True, False)
        tag = self.crf_tag.decode(tag, mask = mask)

        return tag

In [7]:
total_tags = []
with open(data_folder + "/classes.txt") as f:
    for line in f.readlines():
        total_tags.append(line.strip())
        

enc_tag = preprocessing.LabelEncoder()
enc_tag.fit(list(total_tags))

def process_data(data_path):
    sentences, tags = [], []
    sentence, tag = [], []
    
    total_tags = set()
    i = 0
    
    for path in data_path:
        with open(path, "r") as f:
            for line in f:
                if i % 10000 == 0:
                    print(len(sentences))
                
                i += 1
                line = line.strip()
                if line.startswith("-DOCSTART-"):
                    continue
                elif len(line) == 0:
                    if sentence == [] and tag == []:
                        continue
                        
                    sentences.append(sentence)
                    tags.append(tag)
                    sentence, tag = [], []
                else:
                    s,t = line.split("\t")
                    sentence.append(s)
                    tag.append(t)
                    
    for i in range(len(tags)):
        tags[i] = enc_tag.transform(tags[i])
        
    return sentences, tags, enc_tag

In [8]:
# Annotation Tagging 

def split_tag(chunk_tag):
    """
    split chunk tag into IOBES prefix and chunk_type
    e.g.
    B-PER --> (B, PER)
    O --> (0, None)
    """
    if chunk_tag == 'O':
        return ('O', None)
    return chunk_tag.split("-", maxsplit = 1)

def is_chunk_end(prev_tag, tag):
    """
    check if the previous chunk ended between the previous and current word
    e.g.
    (B-PER, I-PER) --> False
    (B-LOC, O) --> True
    Note: in case of contradicting tags, e.g. (B-PER, I-LOC)
    this is considered as (B-PER, B-LOC)
    """
    
    prefix1, chunk_type1 = split_tag(prev_tag)
    prefix2, chunk_type2 = split_tag(tag)
    
    if prefix1 == 'O':
        return False
    if prefix2 == 'O':
        return prefix1 != 'O'
    
    if chunk_type1 != chunk_type2:
        return True
    
    return prefix2 in ["B", "S"] or prefix1 in ["E", "S"]

def is_chunk_start(prev_tag, tag):
    """
    check if a new chunk started between the previous and current word
    """
    prefix1, chunk_type1 = split_tag(prev_tag)
    prefix2, chunk_type2 = split_tag(tag)
    
    if prefix2 == "O":
        return False
    if prefix1 == "O":
        return prefix2 != "O"
    
    if chunk_type1 != chunk_type2:
        return True
    
    return prefix2 in ["B", "S"] or prefix1 in ["E", "S"]

In [9]:
def calc_metrics(tp, p, t, percent = True):
    """
    compute overall precision, recall and F1-Score (default values are 0.0)
    if percent is True, return 100 * original decimal value
    """
    precision = tp / p if p else 0
    recall = tp / t if t else 0
    fb1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
    if percent:
        return 100 * precision, 100 * recall, 100 * fb1
    else:
        return precision, recall, fb1

In [10]:
def count_chunks(true_seqs, pred_seqs):
    
    """
    true_seqs: a list of true tags
    pred_seqs: a list of predicted tags
    return: 
    correct_chunks: a dict (counter), 
                    key = chunk types, 
                    value = number of correctly identified chunks per type
    true_chunks:    a dict, number of true chunks per type
    pred_chunks:    a dict, number of identified chunks per type
    correct_counts, true_counts, pred_counts: similar to above, but for tags
    """
    
    correct_chunks = defaultdict(int)
    true_chunks = defaultdict(int)
    pred_chunks = defaultdict(int)

    correct_counts = defaultdict(int)
    true_counts = defaultdict(int)
    pred_counts = defaultdict(int)

    prev_true_tag, prev_pred_tag = 'O', 'O'
    correct_chunk = None

    for true_tag, pred_tag in zip(true_seqs, pred_seqs):
        if true_tag == pred_tag:
            correct_counts[true_tag] += 1
        true_counts[true_tag] += 1
        pred_counts[pred_tag] += 1

        _, true_type = split_tag(true_tag)
        _, pred_type = split_tag(pred_tag)

        if correct_chunk is not None:
            true_end = is_chunk_end(prev_true_tag, true_tag)
            pred_end = is_chunk_end(prev_pred_tag, pred_tag)

            if pred_end and true_end:
                correct_chunks[correct_chunk] += 1
                correct_chunk = None
            elif pred_end != true_end or true_type != pred_type:
                correct_chunk = None

        true_start = is_chunk_start(prev_true_tag, true_tag)
        pred_start = is_chunk_start(prev_pred_tag, pred_tag)

        if true_start and pred_start and true_type == pred_type:
            correct_chunk = true_type
        if true_start:
            true_chunks[true_type] += 1
        if pred_start:
            pred_chunks[pred_type] += 1

        prev_true_tag, prev_pred_tag = true_tag, pred_tag
    if correct_chunk is not None:
        correct_chunks[correct_chunk] += 1

    return (correct_chunks, true_chunks, pred_chunks, 
        correct_counts, true_counts, pred_counts)

In [11]:
# Print the results
def get_result(correct_chunks, true_chunks, pred_chunks,
    correct_counts, true_counts, pred_counts, verbose = True):
    
    """
    if verbose, print overall performance, as well as preformance per chunk type;
    otherwise, simply return overall prec, rec, f1 scores
    """
    
    # sum counts
    sum_correct_chunks = sum(correct_chunks.values())
    sum_true_chunks = sum(true_chunks.values())
    sum_pred_chunks = sum(pred_chunks.values())

    sum_correct_counts = sum(correct_counts.values())
    sum_true_counts = sum(true_counts.values())

    nonO_correct_counts = sum(v for k, v in correct_counts.items() if k != 'O')
    nonO_true_counts = sum(v for k, v in true_counts.items() if k != 'O')

    chunk_types = sorted(list(set(list(true_chunks) + list(pred_chunks))))

    # compute overall precision, recall and F1-Score (default values are 0.0)
    prec, rec, f1 = calc_metrics(sum_correct_chunks, sum_pred_chunks, sum_true_chunks)
    res = (prec, rec, f1)
    if not verbose:
        return res

    # print overall performance, and performance per chunk type
    
    print("processed %i tokens with %i phrases; " % (sum_true_counts, sum_true_chunks), end='')
    print("found: %i phrases; correct: %i.\n" % (sum_pred_chunks, sum_correct_chunks), end='')
        
    print("accuracy: %6.2f%%; (non-O)" % (100 * nonO_correct_counts/nonO_true_counts))
    print("accuracy: %6.2f%%; " % (100 * sum_correct_counts/sum_true_counts), end = '')
    print("precision: %6.2f%%; recall: %6.2f%%; F1-Score: %6.2f" % (prec, rec, f1))

    # for each chunk type, compute precision, recall and FB1 (default values are 0.0)
    for t in chunk_types:
        prec, rec, f1 = calc_metrics(correct_chunks[t], pred_chunks[t], true_chunks[t])
        print("%17s: " %t , end = '')
        print("precision: %6.2f%%; recall: %6.2f%%; F1-Score: %6.2f" % (prec, rec, f1), end = '')
        print("  %d" % pred_chunks[t])

    return res

In [12]:
# Evaluate the datasets
def evaluate(true_seqs, pred_seqs, verbose = True):
    (correct_chunks, true_chunks, pred_chunks, correct_counts, true_counts, pred_counts) = count_chunks(true_seqs, pred_seqs)
    result = get_result(correct_chunks, true_chunks, pred_chunks, correct_counts, true_counts, pred_counts, verbose = verbose)
    return result

In [13]:
# Set the train / validation / test set for each loss
def train_fn(data_loader, model, optimizer, device):
    model.train()
    final_loss = 0
    for data in tqdm(data_loader, total = len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        loss = model(**data)
        loss.backward()
        optimizer.step()
        final_loss += loss.item()
    return final_loss / len(data_loader)

def eval_fn(data_loader, model, device):
    model.eval()
    final_loss = 0
    
    for data in tqdm(data_loader, total = len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        loss = model(**data)
        final_loss += loss.item()
    return final_loss / len(data_loader)


def test_fn(dataset,model,device,enc_tag):
    final_test = []
    final_pred = []
    O = enc_tag.transform(["O"])[0]
  
    with torch.no_grad():
        for data in tqdm(dataset):
            for k, v in data.items():
                data[k] = v.to(device).unsqueeze(0)

            tag = model.encode(**data)
            padded_pred = tag[0]
            test = data["target_tag"].cpu()[0][:len(padded_pred)]
            test = enc_tag.inverse_transform(test)
            padded_pred = enc_tag.inverse_transform(padded_pred)
            final_pred.extend(padded_pred[1:-1])
            final_test.extend(test[1:-1])
  
    print(evaluate(final_test, final_pred))

In [14]:
# Load saved model
def load_model(epochs):
    path = model_path + f"_{epochs}.bin"
    device = torch.device(device)
    model = EntityModel(num_tag = num_tag)
    model.load_state_dict(torch.load(path))
    model.to(device)
    return model

In [15]:
# Load the NCBI Disease
if __name__ == "__main__":
    sentences, tag, enc_tag = process_data([data_folder + '/NCBI-disease-IOB/train.tsv',
                                            data_folder + '/NCBI-disease-IOB/dev.tsv'])
    test_sentences, test_tag, _ = process_data([data_folder + '/NCBI-disease-IOB/test.tsv'])

    meta_data = {
        "enc_tag": enc_tag
    }

    joblib.dump(meta_data, "meta.bin")

    num_tag = len(list(enc_tag.classes_))

    (
        train_sentences,
        valid_sentences,
        train_tag,
        valid_tag
    ) = model_selection.train_test_split(sentences, 
                                         tag, 
                                         random_state = 42, 
                                         test_size = 0.1)

    train_dataset = EntityDataset(texts = train_sentences, 
                                  tags = train_tag,
                                  enc_tag = enc_tag)

    train_data_loader = torch.utils.data.DataLoader(train_dataset, 
                                                    batch_size = train_batch_size, 
                                                    num_workers = num_workers)

    valid_dataset = EntityDataset(texts = valid_sentences, 
                                  tags = valid_tag,
                                  enc_tag = enc_tag)

    valid_data_loader = torch.utils.data.DataLoader(valid_dataset, 
                                                    batch_size = valid_batch_size, 
                                                    num_workers = num_workers)

    test_dataset = EntityDataset(texts = test_sentences,
                                 tags = test_tag, 
                                 enc_tag = enc_tag)

    device = torch.device(device)
    model = EntityModel(num_tag = num_tag)
    model.to(device)

    # optimizer = torch.optim.Adam(model.parameters(), lr=3e-5, weight_decay=0.001)
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)


0
385
766
1146
1532
1907
2300
2690
3083
3459
3839
4249
4631
5007
5383
5766
6126
0
369
734


In [16]:
# Run the model!
for epoch in range(epochs):
    train_loss = train_fn(train_data_loader, model, optimizer, device)
    torch.cuda.empty_cache()
    valid_loss = eval_fn(valid_data_loader, model, device)
    torch.cuda.empty_cache()
    print(f"Train Loss = {train_loss}")
    print(f"Validation loss = {valid_loss}")
    test_fn(test_dataset, model, device, enc_tag)
    torch.save(model.state_dict(), model_path + f"_{epoch}.bin")

100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  3.40it/s]


Train Loss = 0.19001987568206258
Validation loss = 0.10474997013807297


100%|█████████████████████████████████████████| 939/939 [00:14<00:00, 64.11it/s]


processed 26587 tokens with 1361 phrases; found: 1376 phrases; correct: 624.
accuracy:  74.32%; (non-O)
accuracy:  95.64%; precision:  45.35%; recall:  45.85%; F1-Score:  45.60
          Disease: precision:  45.35%; recall:  45.85%; F1-Score:  45.60  1376
(45.348837209302324, 45.8486407053637, 45.59736938253562)


100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  3.08it/s]


...

processed 26587 tokens with 1361 phrases; found: 1408 phrases; correct: 1239.
accuracy:  93.37%; (non-O)
accuracy:  98.32%; precision:  88.00%; recall:  91.04%; F1-Score:  89.49
          Disease: precision:  88.00%; recall:  91.04%; F1-Score:  89.49  1408
(87.9971590909091, 91.03600293901543, 89.4907908992416)


100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  3.35it/s]


Train Loss = 0.004520943965892204
Validation loss = 0.060908088274300096


100%|█████████████████████████████████████████| 939/939 [00:14<00:00, 64.13it/s]


processed 26587 tokens with 1361 phrases; found: 1407 phrases; correct: 1229.
accuracy:  93.86%; (non-O)
accuracy:  98.21%; precision:  87.35%; recall:  90.30%; F1-Score:  88.80
          Disease: precision:  87.35%; recall:  90.30%; F1-Score:  88.80  1407
(87.34896943852168, 90.30124908155767, 88.80057803468208)
